<a href="https://colab.research.google.com/github/mansi1189/Natural-Language-Processing/blob/master/ATIS_NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from zipfile import ZipFile
import os
import pickle
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from seqeval.metrics import f1_score


Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
!pip3 install pytorch_pretrained_bert==0.4.0
!pip install seqeval

In [0]:
with ZipFile("atis.zip", 'r') as zipObj:
   zipObj.extractall()


In [0]:
def load_ds(fname='atis.train.pkl'):
    with open(fname, 'rb') as stream:
        ds,dicts = pickle.load(stream)
    print('Done  loading: ', fname)
    print('      samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size: {:4d}'.format(len(dicts['token_ids'])))
    print('   slot count: {:4d}'.format(len(dicts['slot_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))
    return ds,dicts


In [0]:
# my_dict_final = []  # Create an empty dictionary
# with open('atis.train.pkl', 'rb') as f:
#     my_dict_final.append(pickle.load(f))   # Update contents of file1 to the dictionary
# with open('atis.test.pkl', 'rb') as f:
#     my_dict_final.append(pickle.load(f))   # Update contents of file2 to the dictionary
# print(my_dict_final)

In [6]:
train_ds, dicts = load_ds('atis.train.pkl')
test_ds, dicts  = load_ds('atis.test.pkl')

Done  loading:  atis.train.pkl
      samples: 4978
   vocab_size:  943
   slot count:  129
 intent count:   26
Done  loading:  atis.test.pkl
      samples:  893
   vocab_size:  943
   slot count:  129
 intent count:   26


In [0]:
#Python map() function is used to apply a function on all the elements of specified iterable and return map object. Python map object is an iterator, so we can iterate over its elements.
# TRaining
t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids','intent_ids'])
i2t, i2s, i2in = map(lambda d: {d[k]:k for k in d.keys()}, [t2i,s2i,in2i])
#The method get() returns a value for the given key. If key is not available then returns default value None.
query, slots, intent =  map(train_ds.get, ['query', 'slot_labels', 'intent_labels'])

sentence=[]

labels=[]
z=[]

for i in range(len(query)):
  y='{}'.format(' '.join(map(i2t.get, query[i])))
  sentence.append(y)
  lab=[]
  for j in range(len(query[i])):
    z='{}'.format(i2s[slots[i][j]])
#     print(z)  
    lab.append(z)
#   print(lab) 
  labels.append(lab)

# for i in labels:
  
#   print(i)  
  

In [0]:
# TEST


tt2i, ts2i, tin2i = map(dicts.get, ['token_ids', 'slot_ids','intent_ids'])
ti2t, ti2s, ti2in = map(lambda d: {d[k]:k for k in d.keys()}, [tt2i,ts2i,tin2i])
#The method get() returns a value for the given key. If key is not available then returns default value None.
tquery, tslots, tintent =  map(test_ds.get, ['query', 'slot_labels', 'intent_labels'])

tsentence=[]

tlabels=[]
tz=[]

for i in range(len(tquery)):
  y='{}'.format(' '.join(map(ti2t.get, tquery[i])))
  tsentence.append(y)
  tlab=[]
  for j in range(len(tquery[i])):
    tz='{}'.format(ti2s[tslots[i][j]])
#     print(tz)  
    tlab.append(tz)
#     print(tlab) 
  tlabels.append(tlab)

#   print(tlab)
# for i in tlabels:

#   print(i)  

In [0]:
s2i

In [0]:
l=[]
for i in range(len(query)):
  
  for j in range(len(query[i])):
    z='{}'.format(i2s[slots[i][j]])
#     print(z)  
    l.append(z)
#   

In [11]:
tags_vals = list(s2i.keys())
print(tags_vals)

['B-aircraft_code', 'B-airline_code', 'B-airline_name', 'B-airport_code', 'B-airport_name', 'B-arrive_date.date_relative', 'B-arrive_date.day_name', 'B-arrive_date.day_number', 'B-arrive_date.month_name', 'B-arrive_date.today_relative', 'B-arrive_time.end_time', 'B-arrive_time.period_mod', 'B-arrive_time.period_of_day', 'B-arrive_time.start_time', 'B-arrive_time.time', 'B-arrive_time.time_relative', 'B-booking_class', 'B-city_name', 'B-class_type', 'B-compartment', 'B-connect', 'B-cost_relative', 'B-day_name', 'B-day_number', 'B-days_code', 'B-depart_date.date_relative', 'B-depart_date.day_name', 'B-depart_date.day_number', 'B-depart_date.month_name', 'B-depart_date.today_relative', 'B-depart_date.year', 'B-depart_time.end_time', 'B-depart_time.period_mod', 'B-depart_time.period_of_day', 'B-depart_time.start_time', 'B-depart_time.time', 'B-depart_time.time_relative', 'B-economy', 'B-fare_amount', 'B-fare_basis_code', 'B-flight', 'B-flight_days', 'B-flight_mod', 'B-flight_number', 'B-fl

In [12]:
labels[:5]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'O',
  'O',
  'B-toloc.city_name',
  'O',
  'B-arrive_time.time',
  'O',
  'O',
  'B-arrive_time.period_of_day',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O',
  'B-depart_date.day_name',
  'B-depart_time.period_of_day',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'B-flight_time',
  'I-flight_time',
  'O',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'O',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'O',
  'B-fromloc.city_name',
  'O'],
 ['O',
  'B-cost_relative',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O'],
 ['O',
  'B-round_trip',
  'I-round_trip',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'B-cost_relative',
  'B-fare_amount',
  'I-fare_amount',
  'O']]

In [13]:
# Visualization

for i in range(5):
    print('{:4d}:{:>15}: {}'.format(i, i2in[intent[i][0]],
                                    ' '.join(map(i2t.get, query[i]))))
    for j in range(len(query[i])):
        print('{:>33} {:>40}'.format(i2t[query[i][j]],
                                     i2s[slots[i][j]]  ))
    print('*'*74)

   0:         flight: BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS
                              BOS                                        O
                                i                                        O
                             want                                        O
                               to                                        O
                              fly                                        O
                             from                                        O
                           boston                      B-fromloc.city_name
                               at                                        O
                              838                       B-depart_time.time
                               am                       I-depart_time.time
                              and                                        O
                           arrive                                

In [0]:
MAX_LEN = 75
bs = 32

In [0]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()


In [16]:
torch.cuda.get_device_name(0) 

'Tesla T4'

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [18]:
# test
ttokenized_texts = [tokenizer.tokenize(sent) for sent in tsentence]
print(ttokenized_texts[0:10])
len(ttokenized_texts)
print()

[['bo', '##s', 'i', 'would', 'like', 'to', 'find', 'a', 'flight', 'from', 'charlotte', 'to', 'las', 'vegas', 'that', 'makes', 'a', 'stop', 'in', 'st', '.', 'louis', 'e', '##os'], ['bo', '##s', 'on', 'april', 'first', 'i', 'need', 'a', 'ticket', 'from', 'tacoma', 'to', 'san', 'jose', 'departing', 'before', '7', 'am', 'e', '##os'], ['bo', '##s', 'on', 'april', 'first', 'i', 'need', 'a', 'flight', 'going', 'from', 'phoenix', 'to', 'san', 'diego', 'e', '##os'], ['bo', '##s', 'i', 'would', 'like', 'a', 'flight', 'traveling', 'one', 'way', 'from', 'phoenix', 'to', 'san', 'diego', 'on', 'april', 'first', 'e', '##os'], ['bo', '##s', 'i', 'would', 'like', 'a', 'flight', 'from', 'orlando', 'to', 'salt', 'lake', 'city', 'for', 'april', 'first', 'on', 'delta', 'airlines', 'e', '##os'], ['bo', '##s', 'i', 'need', 'a', 'flight', 'from', 'toronto', 'to', 'newark', 'one', 'way', 'leaving', 'wednesday', 'evening', 'or', 'thursday', 'morning', 'e', '##os'], ['bo', '##s', 'monday', 'morning', 'i', 'would

In [0]:
df = pd.DataFrame(sentence)
df.to_csv('list.csv', index=False)

In [20]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]
print(tokenized_texts[0:10])
len(tokenized_texts)

[['bo', '##s', 'i', 'want', 'to', 'fly', 'from', 'boston', 'at', '83', '##8', 'am', 'and', 'arrive', 'in', 'denver', 'at', '111', '##0', 'in', 'the', 'morning', 'e', '##os'], ['bo', '##s', 'what', 'flights', 'are', 'available', 'from', 'pittsburgh', 'to', 'baltimore', 'on', 'thursday', 'morning', 'e', '##os'], ['bo', '##s', 'what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '75', '##5', 'am', 'flight', 'leaving', 'washington', 'e', '##os'], ['bo', '##s', 'cheap', '##est', 'air', '##fare', 'from', 'tacoma', 'to', 'orlando', 'e', '##os'], ['bo', '##s', 'round', 'trip', 'fares', 'from', 'pittsburgh', 'to', 'philadelphia', 'under', '1000', 'dollars', 'e', '##os'], ['bo', '##s', 'i', 'need', 'a', 'flight', 'tomorrow', 'from', 'columbus', 'to', 'minneapolis', 'e', '##os'], ['bo', '##s', 'what', 'kind', 'of', 'aircraft', 'is', 'used', 'on', 'a', 'flight', 'from', 'cleveland', 'to', 'dallas', 'e', '##os'], ['bo', '##s', 'show', 'me', 'the', 'flights', 'from', 'pitt

4978

In [0]:
#pad_sequences is used to ensure that all sequences in a list have the same length. By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence.
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", padding="post")

In [0]:
# test
tinput_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in ttokenized_texts],
                          maxlen=MAX_LEN, dtype="long", padding="post")

In [23]:
tinput_ids[:1]

array([[8945, 2015, 1045, 2052, 2066, 2000, 2424, 1037, 3462, 2013, 5904,
        2000, 5869, 7136, 2008, 3084, 1037, 2644, 1999, 2358, 1012, 3434,
        1041, 2891,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]])

In [0]:
tags = pad_sequences([[s2i.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=s2i["O"], padding="post",dtype="long", truncating="post")

In [25]:
print(tags)

[[128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]
 ...
 [128 128 128 ... 128 128 128]
 [128 128   2 ... 128 128 128]
 [128 128 128 ... 128 128 128]]


In [0]:
ttags = pad_sequences([[ts2i.get(l) for l in lab] for lab in tlabels],
                     maxlen=MAX_LEN, value=ts2i["O"], padding="post",dtype="long", truncating="post")

In [0]:
print(len(sentence))
print(len(labels))
print(len(tags))
print(len(input_ids))

In [0]:
print(len(tsentence))
print(len(tlabels))
print(len(ttags))
print(len(tinput_ids))

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
print(attention_masks[:1])

In [0]:
tattention_masks = [[float(i>0) for i in ii] for ii in tinput_ids]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
len(val_inputs)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
t_inputs = torch.tensor(tinput_ids)
t_tags= torch.tensor(ttags)
t_masks=torch.tensor(tattention_masks)


In [0]:
_#TensorDataset:Dataset wrapping tensors.

#Each sample will be retrieved by indexing tensors along the first dimension.

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)


#Data loader. Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
# sampler (Sampler, optional) – defines the strategy to draw samples from the dataset. 
# bs=32

test_data = TensorDataset(t_inputs, t_masks, t_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)




In [0]:
type(train_data)

In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(s2i))

In [0]:
# model.eval()

In [0]:
model.cuda();

In [0]:

input_ids.shape

In [0]:

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)


In [0]:


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In some cases you may find that each layer of your net amplifies the gradient it receives. This causes a problem because the lower layers of the net then get huge gradients and their updates will be far too large to allow the model to learn anything.

This function ‘clips’ the norm of the gradients by scaling the gradients down by the same amount in order to reduce the norm to an acceptable level. In practice this places a limit on the size of the parameter updates.

The hope is that this will ensure that your model gets reasonably sized gradients and that the corresponding updates will allow the model to learn.

In [38]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    print("tr loss ",tr_loss)
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    print("eval_example ",nb_eval_examples)
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.38045996789421355
tr loss  53.264395505189896


Epoch:  20%|██        | 1/5 [01:21<05:24, 81.06s/it]

eval_example  498
Validation loss: 0.13104051910340786
Validation Accuracy: 0.9710416666666666
F1-Score: 0.5485008818342152
Train loss: 0.09737148960786206
tr loss  13.632008545100689


Epoch:  40%|████      | 2/5 [02:41<04:02, 80.96s/it]

...

eval_example  498
Validation loss: 0.04130183800589293
Validation Accuracy: 0.9907581018518519
F1-Score: 0.8569757238958761
Train loss: 0.03705184916034341
tr loss  5.187258882448077


Epoch:  80%|████████  | 4/5 [05:22<01:20, 80.75s/it]

eval_example  498
Validation loss: 0.0303974577691406
Validation Accuracy: 0.9929484953703704
F1-Score: 0.8937169700528479
Train loss: 0.02563686430055116
tr loss  3.5891610020771623


Epoch: 100%|██████████| 5/5 [06:43<00:00, 80.66s/it]

eval_example  498
Validation loss: 0.025422186066862196
Validation Accuracy: 0.9946788194444444
F1-Score: 0.9199766355140188


In [0]:
print(nb_tr_steps)

In [0]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
#     print(b_input_ids)
    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
#     print("predicted tag: ",label_ids)
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1



In [0]:
#save model
torch.save(model.bert.state_dict(), 'output_model_file')

In [0]:
for p in predictions:
  for p_i in p:
    print("words: {} Tags: {}".format(p_i,tags_vals[p_i]))

In [42]:
pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]

valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]

print('{:>33} {:>40}'.format("Predicted","Actual"  ))                                
for i in range(len(pred_tags)):
  
  print('Predic: {}\n Actual: {}\n\n'.format(pred_tags[i],valid_tags[i] ))
print('*'*74)
print(" loss: {}".format(eval_loss/nb_eval_steps))
print(" Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print(" F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

                        Predicted                                   Actual
Predic: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'B-stoploc.city_name', 'I-stoploc.city_name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
 Actual: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'B-stoploc.city_name', 'I-stoploc.city_name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',